In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import json
import numpy as np

text = []
label = []

with open("../../data/domain1_train.json") as f:
    for line in f:
        # read line by line
        data = json.loads(line)
        
        # add values
        text.append(data["text"])
        label.append(data["label"])

with open("../../data/domain2_train.json") as f:
    for line in f:
        # read line by line
        data = json.loads(line)
        
        # add values
        text.append(data["text"])
        label.append(data["label"])


In [2]:
vector_sample = [np.arange(5000).tolist()]
# vector_sample += text[:100]
# vector_sample += text[-100:]
vector_sample += text
print("instance size:", len(vector_sample))

instance size: 34401


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

def toStr(n):
   return str(n)

# Create a Vectorizer Object
vectorizer = CountVectorizer(preprocessor= toStr, analyzer="word", token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 3), max_features=10000)

vectorizer.fit(vector_sample)

# Encode the Document
vector = vectorizer.transform(text)

X = vector.toarray()
y = np.array(label).ravel()

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (34400, 10000)
Shape of y: (34400,)


In [4]:
from collections import Counter
from numpy.random import RandomState
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC # Numerical & Categorical
from imblearn.over_sampling import SMOTEN # Categorical

# X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0, n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print(f'Original dataset shape {X.shape}')
print(f'Original dataset samples per class {Counter(y)}')

# simulate the 2 last columns to be categorical features
# X[:, -2:] = RandomState(10).randint(0, 4, size=(1000, 2))
sm = SMOTEN(random_state=42) # took 406min(7hours)
# sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print(f'Resampled dataset samples per class {Counter(y_res)}')

Original dataset shape (34400, 10000)
Original dataset samples per class Counter({0: 22500, 1: 11900})


KeyboardInterrupt: 

In [ ]:
dict = {"X_res": X_res.tolist(), "y_res": y_res.tolist()}

In [ ]:
# Serializing json
json_object = json.dumps(dict, indent=4)
 
# Writing to sample.json
with open("data_bal.json", "w") as outfile:
    outfile.write(json_object)